Learn Programmation with Python <a class="anchor" id="Title"></a>
===

## Table of contents

* [Design Patterns](#Intro)
    * [Singleton (Creational)](#Singleton)
        * [Naive Singleton](#Singleton1)
        * [Better Singleton](#Singleton2)
    * [Factory (Creational)](#Factory)
    * [Strategy (Behavioral)](#Strategy)
    * [Observer (Behavioral)](#Observer)
    * [Builder (Creational)](#Builder)
    * [Adapter (Structural)](#Adapter)
    * [State (Behavioral)](#State)  
\
* [Testing Your Code](#Test)

# Design Patterns <a class="anchor" id="Intro"></a>

Design Patterns give some standards solutions to recurrent programming problems. It's like a plan to describe how to build the best solution to a know concepting problem. It will gives you a global architecture for your software, that is easy to reproduce (because each design pattern is well known and documented) and to understand (because it's recurrent).

Three types of Design patterns : 
- Creational (Factory, Singleton, Builder, Prototype...): describes how to instantiate and configure classes and objects
- Structural (Adapter, Decorator...): describes how to organise classes in a larger structure
- Behavioral (Iterator, Mediator, Observer, Strategy...): describes how to organise objects to create an easy collaboration between them

[Back to the top.](#Title)

---
## Singleton (Creational)<a class="anchor" id="Singleton"></a>


    Nom ;
    Description du problème à résoudre ;
    Description de la solution : les éléments de la solution, avec leurs relations. La solution est appelée patron de conception ;
    Conséquences : résultats issus de la solution.


### Quick naive singleton<a class="anchor" id="Singleton1"></a>
Used to limit creation of class to only one object. Make the constructor as "private" to the class. In that way, only class members can access to it and no one else.

![](https://sourcemaking.com/files/v2/content/patterns/singleton1.png?id=76898a985727355c4d68)

In [1]:
# A quick first and naive example with private attributes

class ASingletonClass:
    __instance = False
    __arg = None
    
    def __init__(self, arg=None):
        if not ASingletonClass.__instance:
            ASingletonClass.__instance = True
        ASingletonClass.__arg = arg
        
    @property
    def arg(self):
        return self.__arg
            
    def __getattr__(self, name):
        return getattr(self.instance, name)
        
        
x = ASingletonClass('bouh')
print("Value of X : " + x.arg)
y = ASingletonClass('paf')
print("Value of Y : " + y.arg)
print("Value of X : " + x.arg)

Value of X : bouh
Value of Y : paf
Value of X : paf


Nice try, but not. that the above approach doesn’t restrict you to creating only one object. You don't believe me ? Let's take a look on the object addresses :

In [2]:
print("Address of X : " + str(x))
print("Address of Y : " + str(y))

Address of X : <__main__.ASingletonClass object at 0x000002695C1BD460>
Address of Y : <__main__.ASingletonClass object at 0x000002695C1BD580>


Yep... They aren't equal... So let's try another way ! In python 2.2, add the keyword `__new__`. Where `__init__` just initialize an object, `__new__` creates a new one and so, needs to return the object. 

### Better singleton with Python<a class="anchor" id="Singleton2"></a>
In general, `__new__` is used to override the constructor. Here an example of a singleton with `__new__` :

In [3]:
class ASingletonClass(object):
    __instance = None
    def __new__(cls, val):
        if ASingletonClass.__instance is None:
            ASingletonClass.__instance = object.__new__(cls)
        ASingletonClass.__instance.val = val
        
        return ASingletonClass.__instance
    
    @property
    def arg(self):
        return ASingletonClass.__instance.val
    
    def __getattr__(self, name):
        return getattr(self.__instance, name)

In [4]:
x = ASingletonClass('bouh')
print("Value of X : " + str(x.arg))
y = ASingletonClass('paf')
print("Value of Y : " + str(y.arg))
print("Value of X : " + str(x.arg))
print("Address of Y : " + str(y))
print("Address of X : " + str(x))

Value of X : bouh
Value of Y : paf
Value of X : paf
Address of Y : <__main__.ASingletonClass object at 0x000002695C1BDB50>
Address of X : <__main__.ASingletonClass object at 0x000002695C1BDB50>


Here, we need to override object class' constructor.

[Back to the top.](#Title)

---

## Factory (Creational)<a class="anchor" id="Factory"></a>

A factory produces some goods, here we'll produce some objects, and moreover, without considering its type ! In fact, Factory creates concrete implementations of a common interface Factories are generally unique in the code, so we often use singleton in the same time.  


![](https://sourcemaking.com/files/v2/content/patterns/Factory_Method.png?id=3399879c51b9ed8ee453)

For example, we need an object that can be a Circle, a Rectangle or a Square :

In [5]:
import abc

# Create the abstract method
class Shape(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def calculate_area(self):
        pass

    @abc.abstractmethod
    def calculate_perimeter(self):
        pass


# Create all shape forms
class Rectangle(Shape):  # ConcreteProduct
    def __init__(self, height, width):
        self.height = height
        self.width = width

    def calculate_area(self):
        return self.height * self.width 

    def calculate_perimeter(self):
        return 2 * (self.height + self.width) 

    
class Square(Shape):  # ConcreteProduct
    def __init__(self, width):
        self.width = width

    def calculate_area(self):
        return self.width ** 2

    def calculate_perimeter(self):
        return 4 * self.width

    
class Circle(Shape):  # ConcreteProduct
    def __init__(self, radius):
        self.radius = radius

    def calculate_area(self):
        return 3.14 * self.radius * self.radius

    def calculate_perimeter(self):
        return 2 * 3.14 * self.radius

In [6]:
# Create the factory
class ShapeFactory:  # Creator
    __instance = None
    def __new__(cls):
        if ShapeFactory.__instance is None:
            ShapeFactory.__instance = object.__new__(cls)
        
        return ShapeFactory.__instance
    
    def create_shape(self, name, value1=0, value2=0):
        if name == 'circle':
            radius = value1 #  input("Enter the radius of the circle: ")
            return Circle(float(radius))

        elif name == 'rectangle':
            height = value2 #  input("Enter the height of the rectangle: ")
            width = value1 #  input("Enter the width of the rectangle: ")
            return Rectangle(int(height), int(width))

        elif name == 'square':
            width = value1 #  input("Enter the width of the square: ")
            return Square(int(width))

In [7]:
factory = ShapeFactory()
shape = factory.create_shape('circle', value1=10.0)
print('- Circle -\n   Area : ' + str(shape.calculate_area()) + '\n   Perimeter : ' + str(shape.calculate_perimeter()))

shape = factory.create_shape('rectangle', value1=10.0, value2=1.0)
print('\n\n- Rectangle -\n   Area : ' + str(shape.calculate_area()) + '\n   Perimeter : ' + str(shape.calculate_perimeter()))

shape = factory.create_shape('square', value1=10.0)
print('\n\n- Square -\n   Area : ' + str(shape.calculate_area()) + '\n   Perimeter : ' + str(shape.calculate_perimeter()))

- Circle -
   Area : 314.0
   Perimeter : 62.800000000000004


- Rectangle -
   Area : 10
   Perimeter : 22


- Square -
   Area : 100
   Perimeter : 40


[Back to the top.](#Title)

---

## Strategy (Behavioral) <a class="anchor" id="Strategy"></a>

[Interesting link, click me !](https://sourcemaking.com/design_patterns/strategy)

The strategy pattern allows grouping related algorithms under an abstraction, which allows switching out one algorithm or policy for another without modifying the client. Instead of directly implementing a single algorithm, the code receives runtime instructions specifying which of the group of algorithms to run.

![](https://sourcemaking.com/files/v2/content/patterns/Strategy_.png?id=786363dbbdb04e551aa8)

In [8]:
import abc


# Define the interface of interest to clients.
class Context:
    """    
    Maintain a reference to a Strategy object.
    """

    def __init__(self, strategy):
        self._strategy = strategy

    def context_interface(self):
        self._strategy.algorithm_interface()

In [9]:
# Declare an interface common to all supported algorithms. 
class Strategy(metaclass=abc.ABCMeta):
    """
    Context uses this interface to call the algorithm defined by a
    ConcreteStrategy.
    """
    
    @abc.abstractmethod
    def algorithm_interface(self):
        pass


class Wolf(Strategy):
    """Implement the algorithm using the Strategy interface."""

    def algorithm_interface(self):
        print("Ahouuuuu")

    
class Turtle(Strategy):
    """Implement the algorithm using the Strategy interface."""

    def algorithm_interface(self):
        print("aaahhhhh")

    
class Goat(Strategy):
    """Implement the algorithm using the Strategy interface."""

    def algorithm_interface(self):
        print("beeeeee")

In [10]:
print("\nFirst context:")
my_wolf = Wolf()
context = Context(my_wolf)
context.context_interface()

print("\nChange the context:")
context = Context(Turtle())
context.context_interface()


First context:
Ahouuuuu

Change the context:
aaahhhhh


[Back to the top.](#Title)

---

## Observer (Behavioral) <a class="anchor" id="Observer"></a>

Define a one-to-many dependency between objects so that when one object changes state, all its dependents are notified and updatedautomatically.  

**Problem**  
A client need to be notified when a specific product is available. Two naive solutions exists :
- Firstly, the client can go many times to the shop, but a lot of the time, it is useless as the product isn't available
- A second naive solution is to notify every clients when a new product is available, but the shop will spam all client with a lot of useless messages
In both of these cases, we have a problem, in a case, clients needs a lot of ressources to check the shop, in the other case, the shop spend a lot of ressources to notify (spam) all clients.  

**Solution**  
Some *subscribers* will follow (or unfollow) a *publisher*. The publisher have the list of subscribers and all subscribers can ask to add them or remove them from the list. When a notification arrives to the publisher, it will call the notify function of the list of subscribers.

In [11]:
from __future__ import annotations
import typing
import abc
import numpy as np

class Subject(abc.ABC):  # Publisher interface
    """
    The Subject interface declares a set of methods for managing subscribers.
    """
    
    @abc.abstractmethod
    def attach(self, observer: Observer) -> None:
        """
        Attach an observer to the subject.
        """
        pass

    @abc.abstractmethod
    def detach(self, observer: Observer) -> None:
        """
        Detach an observer from the subject.
        """
        pass

    @abc.abstractmethod
    def notify(self) -> None:
        """
        Notify all observers about an event.
        """
        pass
    
    
class ConcreteSubjectPublisher(Subject):
    """
    The Subject owns some important state and notifies observers when the state
    changes.
    """
    
    # List of subscribers. In real life, the list of subscribers can be stored
    # more comprehensively (categorized by event type, etc.).
    # Use a set unless a list to prevent adding two times the same observer.
    _observers: typing.Set[Observer] = set([])
    # Random state
    _state: int = None
    
    # Subscription management methods.
    def attach(self, observer: Observer) -> None:
        print("Subject: Attached an observer.")
        self._observers.add(observer)

    def detach(self, observer: Observer) -> None:
        print("Subject: Detached an observer.")
        self._observers.remove(observer)
        
    # Trigger an update and notify each Observer
    def notify(self) -> None:
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self)
       
    # Usually, the subscription is just a little part of what the subject does.
    def run_some_logic(self) -> None:
        # ...
        # Do some business
        # ...
        self._state = np.random.randint(6, size=1)
        print("Subject: State = " + str(self._state))
        
        # Trigger notification
        self.notify()

In [12]:
class Observer(abc.ABC):  # Subscriber interface
    """
    The Observer interface declares the update method, used by subjects.
    """

    @abc.abstractmethod
    def update(self, subject: Subject) -> None:
        # Receive update from subject.
        pass

    
class ConcreteObserverA(Observer):
    def update(self, subject: Subject) -> None:
        if subject._state < 3:
            print("ConcreteObserverA: Reacted to the event")


class ConcreteObserverB(Observer):
    def update(self, subject: Subject) -> None:
        if subject._state == 0 or subject._state >= 2:
            print("ConcreteObserverB: Reacted to the event")


class ConcreteObserverC(Observer):
    def update(self, subject: Subject) -> None:
        if subject._state <= 2 or subject._state >= 5:
            print("ConcreteObserverC: Reacted to the event")

In [13]:
# Create the Publisher
subject = ConcreteSubjectPublisher()

# Create and add some observer
observer_a = ConcreteObserverA()
subject.attach(observer_a)

observer_b = ConcreteObserverB()
subject.attach(observer_b)

observer_c = ConcreteObserverC()
subject.attach(observer_c)
 
# Run some logics
for i in range(3):
    subject.run_some_logic()

subject.detach(observer_a)

for i in range(3):
    subject.run_some_logic()

Subject: Attached an observer.
Subject: Attached an observer.
Subject: Attached an observer.
Subject: State = [0]
Subject: Notifying observers...
ConcreteObserverA: Reacted to the event
ConcreteObserverB: Reacted to the event
ConcreteObserverC: Reacted to the event
Subject: State = [4]
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
Subject: State = [4]
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
Subject: Detached an observer.
Subject: State = [2]
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
ConcreteObserverC: Reacted to the event
Subject: State = [3]
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
Subject: State = [5]
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
ConcreteObserverC: Reacted to the event


[Back to the top.](#Title)

---

## Builder (Creational) <a class="anchor" id="Builder"></a>

Separate the construction of a complex object from its representation so that the same construction process can create different representations. It's like a factory but for more complex objects.

In [14]:
from __future__ import annotations
import abc

class Builder(abc.ABC):
    """
    The Builder interface specifies methods for creating the different parts of
    the Product objects.
    """

    @abc.abstractproperty
    def product(self) -> None:
        pass

    @abc.abstractmethod
    def produce_part_a(self) -> None:
        pass

    @abc.abstractmethod
    def produce_part_b(self) -> None:
        pass

    @abc.abstractmethod
    def produce_part_c(self) -> None:
        pass
    

class ConcreteBuilder1(Builder):
    def __init__(self) -> None:
        """
        A fresh builder instance should contain a blank product object, which is
        used in further assembly.
        """
        self.reset()
        
    def reset(self) -> None:
        self._product = Product1()
        
    @property
    def product(self) -> Product1:
        """
        Concrete Builders are supposed to provide their own methods for
        retrieving results. That's because various types of builders may create
        entirely different products that don't follow the same interface.
        Therefore, such methods cannot be declared in the base Builder interface
        (at least in a statically typed programming language).

        Usually, after returning the end result to the client, a builder
        instance is expected to be ready to start producing another product.
        That's why it's a usual practice to call the reset method at the end of
        the `getProduct` method body. However, this behavior is not mandatory,
        and you can make your builders wait for an explicit reset call from the
        client code before disposing of the previous result.
        """
        product = self._product
        self.reset()
        return product

    def produce_part_a(self) -> None:
        self._product.add("PartA1")

    def produce_part_b(self) -> None:
        self._product.add("PartB1")

    def produce_part_c(self) -> None:
        self._product.add("PartC1")

In [15]:
class Product1():
    """
    It makes sense to use the Builder pattern only when your products are quite
    complex and require extensive configuration.

    Unlike in other creational patterns, different concrete builders can produce
    unrelated products. In other words, results of various builders may not
    always follow the same interface.
    """

    def __init__(self) -> None:
        self.parts = []

    def add(self, part: Any) -> None:
        self.parts.append(part)

    def list_parts(self) -> None:
        print(f"Product parts: {', '.join(self.parts)}", end="")

In [16]:
class Director:
    """
    The Director is only responsible for executing the building steps in a
    particular sequence. It is helpful when producing products according to a
    specific order or configuration. Strictly speaking, the Director class is
    optional, since the client can control builders directly.
    """

    def __init__(self) -> None:
        self._builder = None

    @property
    def builder(self) -> Builder:
        return self._builder

    @builder.setter
    def builder(self, builder: Builder) -> None:
        """
        The Director works with any builder instance that the client code passes
        to it. This way, the client code may alter the final type of the newly
        assembled product.
        """
        self._builder = builder

    """
    The Director can construct several product variations using the same
    building steps.
    """

    def build_minimal_viable_product(self) -> None:
        self.builder.produce_part_a()

    def build_full_featured_product(self) -> None:
        self.builder.produce_part_a()
        self.builder.produce_part_b()
        self.builder.produce_part_c()

In [17]:
director = Director()
builder = ConcreteBuilder1()
director.builder = builder

print("Standard basic product: ")
director.build_minimal_viable_product()
builder.product.list_parts()

print("\n")

print("Standard full featured product: ")
director.build_full_featured_product()
builder.product.list_parts()

print("\n")

# Remember, the Builder pattern can be used without a Director class.
print("Custom product: ")
builder.produce_part_a()
builder.produce_part_b()
builder.product.list_parts()

Standard basic product: 
Product parts: PartA1

Standard full featured product: 
Product parts: PartA1, PartB1, PartC1

Custom product: 
Product parts: PartA1, PartB1

[Back to the top.](#Title)

---

## Adapter (Structural) <a class="anchor" id="Adapter"></a>

([A good source !](https://refactoring.guru/fr/design-patterns/adapter))

Convert the interface of a class into another interface clients expect. Adapter lets classes work together that couldn't otherwise because of incompatible interfaces.

[Back to the top.](#Title)

---

## State (Behavioral) <a class="anchor" id="State"></a>

Allow an object to alter its behavior when its internal state changes. The object will appear to change its class.
> An object-oriented state machine  
> wrapper + polymorphic wrappee + collaboration

![](https://sourcemaking.com/files/v2/content/patterns/State1.png?id=04f059639115e34a7628)

In [18]:
import abc


# Define the context class which keeps an instance of State subclass
class Context:
    """
    Maintain an instance of a ConcreteState subclass that defines the
    current state.
    """
    
    _state = None
    
    def __init__(self, state:State):
        self.transition_to(state)
        
    def transition_to(self, state: State):
        self._state = state
        self._state.set_context(self)
        
    def request(self):
        self._state.handle()

In [19]:
# Define main abstract State class
class State(metaclass=abc.ABCMeta):
    """
    Define an interface for encapsulating the behavior associated with a
    particular state of the Context.
    """
    def get_context(self) -> Context:
        return self._context

    def set_context(self, context: Context) -> None:
        self._context = context

    @abc.abstractmethod
    def handle(self):
        pass
    
    
# Define all substates heritate from State   
class ConcreteStateA(State):
    """
    Implement a behavior associated with a state of the Context.
    """

    def handle(self):
        print("Change state A -> B")
        self.get_context().transition_to(ConcreteStateB())


class ConcreteStateB(State):
    """
    Implement a behavior associated with a state of the Context.
    """

    def handle(self):
        print("Change state B -> A")
        self.get_context().transition_to(ConcreteStateA())

In [20]:
context = Context(ConcreteStateA())
context.request()
context.request()
context.request()
context.request()
context.request()

Change state A -> B
Change state B -> A
Change state A -> B
Change state B -> A
Change state A -> B


This example is used for an ON/OFF button. First push, light is on, next tap, light is off, et cetera. But it can be also interesting to switch only if trigger certain request.  

Here, an example with many handle() / request()

In [21]:
import abc


# Define the context class which keeps an instance of State subclass
class Context:
    """
    Maintain an instance of a ConcreteState subclass that defines the
    current state.
    """
    
    _state = None
    
    def __init__(self, state:State):
        self.transition_to(state)
        
    def transition_to(self, state: State):
        self._state = state
        self._state.set_context(self)
        
    def request1(self):
        self._state.handle1()

    def request2(self):
        self._state.handle2()
        
        
# Define main abstract State class
class State(metaclass=abc.ABCMeta):
    """
    Define an interface for encapsulating the behavior associated with a
    particular state of the Context.
    """
    def get_context(self) -> Context:
        return self._context

    def set_context(self, context: Context) -> None:
        self._context = context

    @abc.abstractmethod
    def handle1(self):
        pass
    @abc.abstractmethod
    def handle2(self):
        pass
    
    
# Define all substates heritate from State   
class ConcreteStateA(State):
    """
    Implement a behavior associated with a state of the Context.
    """

    def handle1(self):
        print("A : trigger request 1")
        print("A request bhange state to B")
        self.get_context().transition_to(ConcreteStateB())

    def handle2(self):
        print("A : trigger request 2")
        

class ConcreteStateB(State):
    """
    Implement a behavior associated with a state of the Context.
    """

    def handle1(self):
        print("B : trigger request 1")
        
    def handle2(self):
        print("B : trigger request 2")
        print("B request bhange state to A")
        self.get_context().transition_to(ConcreteStateA())

    
context = Context(ConcreteStateA())
context.request1()
context.request1()
context.request2()
context.request1()
context.request2()
context.request2()
context.request2()
context.request1()

A : trigger request 1
A request bhange state to B
B : trigger request 1
B : trigger request 2
B request bhange state to A
A : trigger request 1
A request bhange state to B
B : trigger request 2
B request bhange state to A
A : trigger request 2
A : trigger request 2
A : trigger request 1
A request bhange state to B


[Back to the top.](#Title)

---

## Testing Your Code  <a class="anchor" id="Test"></a>

Testing your code is very important.

Getting used to writing testing code and running this code in parallel is now considered a good habit. Used wisely, this method helps to define your code’s intent more precisely and have a more decoupled architecture.

Some general rules of testing:

- A testing unit should focus on one tiny bit of functionality and prove it correct.
- Each test unit must be fully independent. Each test must be able to run alone, and also within the test suite, regardless of the order that they are called. The implication of this rule is that each test must be loaded with a fresh dataset and may have to do some cleanup afterwards. This is usually handled by setUp() and tearDown() methods.
- Try hard to make tests that run fast. If one single test needs more than a few milliseconds to run, development will be slowed down or the tests will not be run as often as is desirable. In some cases, tests can’t be fast because they need a complex data structure to work on, and this data structure must be loaded every time the test runs. Keep these heavier tests in a separate test suite that is run by some scheduled task, and run all other tests as often as needed.
- Learn your tools and learn how to run a single test or a test case. Then, when developing a function inside a module, run this function’s tests frequently, ideally automatically when you save the code.
- Always run the full test suite before a coding session, and run it again after. This will give you more confidence that you did not break anything in the rest of the code.
- It is a good idea to implement a hook that runs all tests before pushing code to a shared repository.
- If you are in the middle of a development session and have to interrupt your work, it is a good idea to write a broken unit test about what you want to develop next. When coming back to work, you will have a pointer to where you were and get back on track faster.
- The first step when you are debugging your code is to write a new test pinpointing the bug. While it is not always possible to do, those bug catching tests are among the most valuable pieces of code in your project.
- Use long and descriptive names for testing functions. The style guide here is slightly different than that of running code, where short names are often preferred. The reason is testing functions are never called explicitly. square() or even sqr() is ok in running code, but in testing code you would have names such as test_square_of_number_2(), test_square_negative_number(). These function names are displayed when a test fails, and should be as descriptive as possible.
- When something goes wrong or has to be changed, and if your code has a good set of tests, you or other maintainers will rely largely on the testing suite to fix the problem or modify a given behavior. Therefore the testing code will be read as much as or even more than the running code. A unit test whose purpose is unclear is not very helpful in this case.
- Another use of the testing code is as an introduction to new developers. When someone will have to work on the code base, running and reading the related testing code is often the best thing that they can do to start. They will or should discover the hot spots, where most difficulties arise, and the corner cases. If they have to add some functionality, the first step should be to add a test to ensure that the new functionality is not already a working path that has not been plugged into the interface.

In [22]:
# With Unittest :

import unittest

def fct(x):
    return x + 1

class MyTest(unittest.TestCase):
    def run(self):
        print("Run all tests :")
        self.test()
        print("\nRun all test is finished !")
        
    def test(self):
        print(" - test fct()")
        self.assertEqual(fct(3), 4)

MyTest().run()

Run all tests :
 - test fct()

Run all test is finished !


```python
# With Doctest

import doctest

def int_square_fct(x):
    """Return the square of x.
    
    >>> int_square_fct(2)
    4
    >>> int_square_fct(-2)
    4
    """

    return x * x

doctest.testmod()
```

[Back to the top.](#Title)

---

Thibault **Santonja**  
2021